In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
half = "H2"
### CHOOSE ONE
### H1, H2, None

In [3]:
with open("../matches/matches_England.json", "r") as file:
    match_data_2 = json.load(file)

df2 = pd.DataFrame(match_data_2)
if half == "H1":
    df2 = df2[df2.gameweek<20]
elif half == "H2":
    df2 = df2[df2.gameweek>=20]

with open("../matches/matches_France.json", "r") as file:
    match_data_3 = json.load(file)

df3 = pd.DataFrame(match_data_3)
if half == "H1":
    df3 = df3[df3.gameweek<20]
elif half == "H2":
    df3 = df3[df3.gameweek>=20]


with open("../matches/matches_Germany.json", "r") as file:
    match_data_4 = json.load(file)

df4 = pd.DataFrame(match_data_4)
if half == "H1":
    df4 = df4[df4.gameweek<18]
elif half == "H2":
    df4 = df4[df4.gameweek>=18]

with open("../matches/matches_Italy.json", "r") as file:
    match_data_5 = json.load(file)

df5 = pd.DataFrame(match_data_5)
if half == "H1":
    df5 = df5[df5.gameweek<20]
elif half == "H2":
    df5 = df5[df5.gameweek>=20]

with open("../matches/matches_Spain.json", "r") as file:
    match_data_6 = json.load(file)

df6 = pd.DataFrame(match_data_6)
if half == "H1":
    df6 = df6[df6.gameweek<20]
elif half == "H2":
    df6 = df6[df6.gameweek>=20]

all_matches_df = pd.concat([df2, df3, df4, df5, df6], ignore_index=True)
all_matches_list = list(np.unique(all_matches_df.wyId))

In [4]:
with open("../events/events_England.json", "r") as file:
    event_data_2 = json.load(file)

df2 = pd.DataFrame(event_data_2)
df2 = df2[df2.matchId.isin(all_matches_list)]

with open("../events/events_France.json", "r") as file:
    event_data_3 = json.load(file)

df3 = pd.DataFrame(event_data_3)
df3 = df3[df3.matchId.isin(all_matches_list)]


with open("../events/events_Germany.json", "r") as file:
    event_data_4 = json.load(file)

df4 = pd.DataFrame(event_data_4)
df4 = df4[df4.matchId.isin(all_matches_list)]


with open("../events/events_Italy.json", "r") as file:
    event_data_5 = json.load(file)

df5 = pd.DataFrame(event_data_5)
df5 = df5[df5.matchId.isin(all_matches_list)]


with open("../events/events_Spain.json", "r") as file:
    event_data_6 = json.load(file)

df6 = pd.DataFrame(event_data_6)
df6 = df6[df6.matchId.isin(all_matches_list)]

with open("../events/teams.json", "r") as file:
    teams_data = json.load(file)

teams_df = pd.DataFrame(teams_data)

In [5]:
all_events_df = pd.concat([df2, df3, df4, df5, df6], ignore_index=True)

# Feature engineering - Pass DF - Pass Motifs

In [6]:
passes_df = all_events_df[all_events_df['eventName'] == 'Pass']

# Concatenating teamId and matchId to create a unique identifier for each team in each match
passes_df['uniqueTeamId'] = passes_df['teamId'].astype(str) + "---" + passes_df['matchId'].astype(str)

<ipython-input-6-846069bd07cf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['uniqueTeamId'] = passes_df['teamId'].astype(str) + "---" + passes_df['matchId'].astype(str)


In [7]:
# Function to classify pass motifs
def classify_pass_motif(passes):
    players = passes['playerId'].values
    if len(set(players)) == 2:  # ABAB
        return 'ABAB'
    elif len(set(players)) == 3:
        if players[0] == players[2] or players[1] == players[3]:  # ABCA or ABCB
            return 'ABCA' if players[0] == players[2] else 'ABCB'
        else:
            return 'ABCD'  # ABCD for distinct players or other scenarios
    return 'ABCD'

# Apply the classification within a rolling window
def apply_motif_classification(df):
    # Ensure the DataFrame is sorted by event time
    df = df.sort_values(by='eventSec')
    motifs = []
    # Using a rolling window of 4 events
    for i in range(len(df) - 3):
        window = df.iloc[i:i+4]
        motif = classify_pass_motif(window)
        motifs.append((window.iloc[-1]['uniqueTeamId'], motif))
    return pd.DataFrame(motifs, columns=['uniqueTeamId', 'Motif'])

In [8]:
motif_classification_results = passes_df.groupby('uniqueTeamId').apply(apply_motif_classification)

In [9]:
motif_classification_results.reset_index(drop=True, inplace=True)

In [10]:
motif_counts = motif_classification_results.groupby(['uniqueTeamId', 'Motif']).size().unstack(fill_value=0)

In [11]:
motif_ratios = motif_counts.div(motif_counts.sum(axis=1), axis=0)
# df to be used in clustering

In [12]:
motif_ratios

Motif,ABAB,ABCA,ABCB,ABCD
uniqueTeamId,,,,
10531---2499913,0.030369,0.123644,0.121475,0.724512
10531---2499927,0.018100,0.101810,0.104072,0.776018
10531---2499937,0.024138,0.096552,0.100000,0.779310
10531---2499945,0.043860,0.103070,0.100877,0.752193
10531---2499957,0.022989,0.065134,0.065134,0.846743
...,...,...,...,...
756---2565880,0.029213,0.110112,0.112360,0.748315
756---2565892,0.036842,0.113158,0.105263,0.744737
756---2565904,0.031496,0.089239,0.089239,0.790026


In [13]:
motif_stats = motif_ratios.agg(['mean', 'std']).T

In [14]:
motif_stats

,mean,std
Motif,,
ABAB,0.034334,0.011544
ABCA,0.092703,0.014794
ABCB,0.093317,0.014851
ABCD,0.779646,0.032595


# Feature engineering - Shot DF

In [15]:
shot_df = all_events_df[all_events_df['eventName'] == 'Shot']
shot_df['uniqueTeamId'] = shot_df['teamId'].astype(str) + "---" + shot_df['matchId'].astype(str)

<ipython-input-15-396a5f5d4462>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['uniqueTeamId'] = shot_df['teamId'].astype(str) + "---" + shot_df['matchId'].astype(str)


In [16]:
import math

def calculate_shot_distance(row):
    if row['positions'] is not None and len(row['positions']) > 0:
        loc = row['positions'][0]
        # Normalize locations
        x = loc['x'] * (120 / 100.0)  # Normalizing the x coordinate
        y = loc['y'] * (80 / 100.0)   # Normalizing the y coordinate
        # Calculate Euclidean distance from (0,0)
        distance = math.sqrt((120-x)**2 + (80-y)**2)
        return distance
    return None

# Apply the function to calculate shot distance and add it to the DataFrame
shot_df['shot_distance'] = shot_df.apply(calculate_shot_distance, axis=1)

<ipython-input-16-103ed8455f10>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['shot_distance'] = shot_df.apply(calculate_shot_distance, axis=1)


In [17]:
def classify_shot_by_distance(distance):
    if distance <= 38:
        return "Near"
    elif distance <= 52:
        return "Middle Distance"
    else:
        return "Far"

# Assuming shot_df has a 'shot_distance' column calculated as before
shot_df['shot_class'] = shot_df['shot_distance'].apply(classify_shot_by_distance)

# Grouping shots by their class and counting
shot_class_counts = shot_df.groupby('uniqueTeamId')['shot_class'].value_counts().unstack(fill_value=0)
shot_counts = shot_df.groupby('uniqueTeamId')['subEventName'].value_counts().unstack(fill_value=0)

<ipython-input-17-0451b5045683>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['shot_class'] = shot_df['shot_distance'].apply(classify_shot_by_distance)


In [18]:
shot_df_final = pd.concat([shot_counts, shot_class_counts], axis=1)

In [19]:
shot_df_final

,Shot,Far,Middle Distance,Near
uniqueTeamId,,,,
10531---2499913,6,1,2,3
10531---2499927,9,1,6,2
10531---2499937,7,3,0,4
10531---2499945,7,0,5,2
10531---2499957,3,0,2,1
...,...,...,...,...
756---2565880,19,4,8,7
756---2565892,11,6,3,2
756---2565904,14,0,9,5


In [20]:
shot_df_final.rename(columns = {'Shot':'Total Shots'}, inplace = True) 

shot_df_final["Far"] = shot_df_final["Far"]/shot_df_final["Total Shots"]
shot_df_final["Middle Distance"] = shot_df_final["Middle Distance"]/shot_df_final["Total Shots"]
shot_df_final["Near"] = shot_df_final["Near"]/shot_df_final["Total Shots"]


shot_df_final.rename(columns = {'Near':'Near Shots Ratio', 'Middle Distance':'Middle Distance Shots Ratio', 'Far':'Far Shots Ratio',}, inplace = True) 

In [21]:
shot_df_final

,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio,Near Shots Ratio
uniqueTeamId,,,,
10531---2499913,6,0.166667,0.333333,0.500000
10531---2499927,9,0.111111,0.666667,0.222222
10531---2499937,7,0.428571,0.000000,0.571429
10531---2499945,7,0.000000,0.714286,0.285714
10531---2499957,3,0.000000,0.666667,0.333333
...,...,...,...,...
756---2565880,19,0.210526,0.421053,0.368421
756---2565892,11,0.545455,0.272727,0.181818
756---2565904,14,0.000000,0.642857,0.357143


# Feature engineering - Pass DF - Pass Types

In [22]:
def determine_zone(position):
    if 'x' in position and 'y' in position:
        x, y = position['x'], position['y']
        zone = int(x // 33.333) + 1
        return zone
    else:
        return None
    
# Apply the function to determine zones for each event
passes_df['Zone'] = passes_df['positions'].apply(lambda pos: determine_zone(pos[0]) if len(pos) > 0 else None)

<ipython-input-22-1dcee42e8a09>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['Zone'] = passes_df['positions'].apply(lambda pos: determine_zone(pos[0]) if len(pos) > 0 else None)


In [23]:
passes_df.loc[passes_df['Zone'] == 4, ['Zone']] = 3

In [24]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone
0,8,Simple pass,[{'id': 1801}],20857,"[{'y': 51, 'x': 51}, {'y': 47, 'x': 32}]",2499738,Pass,1619,1H,2.150021,85,238886803,1619---2499738,2
1,8,High pass,[{'id': 1801}],8136,"[{'y': 47, 'x': 32}, {'y': 18, 'x': 79}]",2499738,Pass,1619,1H,4.405917,83,238886804,1619---2499738,1
4,8,Head pass,[{'id': 1801}],37725,"[{'y': 68, 'x': 23}, {'y': 69, 'x': 35}]",2499738,Pass,1633,1H,9.896286,82,238886834,1633---2499738,1
5,8,Head pass,[{'id': 1801}],8677,"[{'y': 69, 'x': 35}, {'y': 67, 'x': 41}]",2499738,Pass,1633,1H,10.893120,82,238886836,1633---2499738,2
6,8,Simple pass,[{'id': 1801}],14703,"[{'y': 67, 'x': 41}, {'y': 70, 'x': 39}]",2499738,Pass,1633,1H,11.644449,85,238886838,1633---2499738,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545241,8,Hand pass,[{'id': 1801}],3486,"[{'y': 29, 'x': 8}, {'y': 19, 'x': 14}]",2565927,Pass,682,2H,2925.547631,81,253302662,682---2565927,1
1545243,8,Simple pass,[{'id': 1801}],3695,"[{'y': 34, 'x': 36}, {'y': 52, 'x': 45}]",2565927,Pass,682,2H,2931.782904,85,253302665,682---2565927,2
1545245,8,Simple pass,[{'id': 1801}],20623,"[{'y': 25, 'x': 66}, {'y': 2, 'x': 88}]",2565927,Pass,682,2H,2939.077491,85,253302671,682---2565927,2
1545247,8,Cross,"[{'id': 401}, {'id': 1802}]",122832,"[{'y': 21, 'x': 97}, {'y': 26, 'x': 92}]",2565927,Pass,682,2H,2942.098761,80,253302674,682---2565927,3


In [25]:
import math

def calculate_pass_details_from_single_column(row):
    if row['positions'] is not None and len(row['positions']) == 2:
        # Extract and normalize start and end locations
        start_loc = row['positions'][0]
        end_loc = row['positions'][1]
        
        start_x = start_loc['x'] * (120 / 100.0)
        start_y = start_loc['y'] * (80 / 100.0)
        end_x = end_loc['x'] * (120 / 100.0)
        end_y = end_loc['y'] * (80 / 100.0)
        
        # Calculate pass length and angle
        delta_x = end_x - start_x
        delta_y = end_y - start_y
        pass_length = math.sqrt(delta_x**2 + delta_y**2)
        pass_angle = math.atan2(delta_y, delta_x)
        
        return pd.Series([pass_length, pass_angle])
    else:
        return pd.Series([None, None])

In [26]:
passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)


<ipython-input-26-53702553e163>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)
<ipython-input-26-53702553e163>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)


In [27]:
def determine_pass_direction(angle):
    # Forward Pass: -π/4 < angle < π/4
    if -np.pi/4 < angle < np.pi/4:
        return "Forward Pass"
    # Backward Pass: 3π/4 < angle <= π or -π <= angle < -3π/4
    elif 3*np.pi/4 < angle <= np.pi or -np.pi <= angle < -3*np.pi/4:
        return "Backward Pass"
    # Side Pass: Otherwise
    else:
        return "Side Pass"

In [28]:
passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)

<ipython-input-28-a0b0c4f17a64>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)


In [29]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone,calculated_pass_length,calculated_pass_angle,pass_direction
0,8,Simple pass,[{'id': 1801}],20857,"[{'y': 51, 'x': 51}, {'y': 47, 'x': 32}]",2499738,Pass,1619,1H,2.150021,85,238886803,1619---2499738,2,23.023466,-3.002153,Backward Pass
1,8,High pass,[{'id': 1801}],8136,"[{'y': 47, 'x': 32}, {'y': 18, 'x': 79}]",2499738,Pass,1619,1H,4.405917,83,238886804,1619---2499738,1,60.985244,-0.390250,Forward Pass
4,8,Head pass,[{'id': 1801}],37725,"[{'y': 68, 'x': 23}, {'y': 69, 'x': 35}]",2499738,Pass,1633,1H,9.896286,82,238886834,1633---2499738,1,14.422205,0.055499,Forward Pass
5,8,Head pass,[{'id': 1801}],8677,"[{'y': 69, 'x': 35}, {'y': 67, 'x': 41}]",2499738,Pass,1633,1H,10.893120,82,238886836,1633---2499738,2,7.375636,-0.218669,Forward Pass
6,8,Simple pass,[{'id': 1801}],14703,"[{'y': 67, 'x': 41}, {'y': 70, 'x': 39}]",2499738,Pass,1633,1H,11.644449,85,238886838,1633---2499738,2,3.394113,2.356194,Side Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545241,8,Hand pass,[{'id': 1801}],3486,"[{'y': 29, 'x': 8}, {'y': 19, 'x': 14}]",2565927,Pass,682,2H,2925.547631,81,253302662,682---2565927,1,10.762899,-0.837981,Side Pass
1545243,8,Simple pass,[{'id': 1801}],3695,"[{'y': 34, 'x': 36}, {'y': 52, 'x': 45}]",2565927,Pass,682,2H,2931.782904,85,253302665,682---2565927,2,18.000000,0.927295,Side Pass
1545245,8,Simple pass,[{'id': 1801}],20623,"[{'y': 25, 'x': 66}, {'y': 2, 'x': 88}]",2565927,Pass,682,2H,2939.077491,85,253302671,682---2565927,2,32.179497,-0.608689,Forward Pass
1545247,8,Cross,"[{'id': 401}, {'id': 1802}]",122832,"[{'y': 21, 'x': 97}, {'y': 26, 'x': 92}]",2565927,Pass,682,2H,2942.098761,80,253302674,682---2565927,3,7.211103,2.553590,Backward Pass


In [30]:
np.unique(passes_df.subEventName)

array(['Cross', 'Hand pass', 'Head pass', 'High pass', 'Launch',
       'Simple pass', 'Smart pass'], dtype=object)

In [31]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone,calculated_pass_length,calculated_pass_angle,pass_direction
0,8,Simple pass,[{'id': 1801}],20857,"[{'y': 51, 'x': 51}, {'y': 47, 'x': 32}]",2499738,Pass,1619,1H,2.150021,85,238886803,1619---2499738,2,23.023466,-3.002153,Backward Pass
1,8,High pass,[{'id': 1801}],8136,"[{'y': 47, 'x': 32}, {'y': 18, 'x': 79}]",2499738,Pass,1619,1H,4.405917,83,238886804,1619---2499738,1,60.985244,-0.390250,Forward Pass
4,8,Head pass,[{'id': 1801}],37725,"[{'y': 68, 'x': 23}, {'y': 69, 'x': 35}]",2499738,Pass,1633,1H,9.896286,82,238886834,1633---2499738,1,14.422205,0.055499,Forward Pass
5,8,Head pass,[{'id': 1801}],8677,"[{'y': 69, 'x': 35}, {'y': 67, 'x': 41}]",2499738,Pass,1633,1H,10.893120,82,238886836,1633---2499738,2,7.375636,-0.218669,Forward Pass
6,8,Simple pass,[{'id': 1801}],14703,"[{'y': 67, 'x': 41}, {'y': 70, 'x': 39}]",2499738,Pass,1633,1H,11.644449,85,238886838,1633---2499738,2,3.394113,2.356194,Side Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545241,8,Hand pass,[{'id': 1801}],3486,"[{'y': 29, 'x': 8}, {'y': 19, 'x': 14}]",2565927,Pass,682,2H,2925.547631,81,253302662,682---2565927,1,10.762899,-0.837981,Side Pass
1545243,8,Simple pass,[{'id': 1801}],3695,"[{'y': 34, 'x': 36}, {'y': 52, 'x': 45}]",2565927,Pass,682,2H,2931.782904,85,253302665,682---2565927,2,18.000000,0.927295,Side Pass
1545245,8,Simple pass,[{'id': 1801}],20623,"[{'y': 25, 'x': 66}, {'y': 2, 'x': 88}]",2565927,Pass,682,2H,2939.077491,85,253302671,682---2565927,2,32.179497,-0.608689,Forward Pass
1545247,8,Cross,"[{'id': 401}, {'id': 1802}]",122832,"[{'y': 21, 'x': 97}, {'y': 26, 'x': 92}]",2565927,Pass,682,2H,2942.098761,80,253302674,682---2565927,3,7.211103,2.553590,Backward Pass


In [32]:
#passes_count = passes_df.groupby('teamId')['subEventName'].value_counts().unstack(fill_value=0)

In [33]:
# Step 1: Combine 'pass_direction' and 'Zone' into a single column
passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)

# Step 2: Group by 'uniqueTeamId' and the new 'dir_zone' column, then count
direction_zone_count = passes_df.groupby(['uniqueTeamId', 'dir_zone']).size()

# Step 3: Pivot the results to have 'uniqueTeamId' as rows and each unique 'dir_zone' combination as columns
direction_zone_count_pivot = direction_zone_count.unstack(fill_value=0)

direction_count = passes_df.groupby('uniqueTeamId')['pass_direction'].value_counts().unstack(fill_value=0)

pass_direction_df = pd.concat([direction_zone_count_pivot, direction_count], axis=1)

<ipython-input-33-54e3dcb6a0f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)


In [34]:
pass_direction_df

,Backward Pass_Zone_1,Backward Pass_Zone_2,Backward Pass_Zone_3,Forward Pass_Zone_1,Forward Pass_Zone_2,Forward Pass_Zone_3,Side Pass_Zone_1,Side Pass_Zone_2,Side Pass_Zone_3,Backward Pass,Forward Pass,Side Pass
uniqueTeamId,,,,,,,,,,,,
10531---2499913,17,57,10,79,60,14,88,103,36,84,153,227
10531---2499927,13,38,11,76,78,5,89,108,27,62,159,224
10531---2499937,15,23,8,58,41,7,64,62,15,46,106,141
10531---2499945,15,50,22,62,74,19,69,107,41,87,155,217
10531---2499957,8,24,12,46,37,10,63,40,24,44,93,127
...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565880,10,30,22,51,66,24,71,100,74,62,141,245
756---2565892,10,31,13,53,64,18,54,92,48,54,135,194
756---2565904,19,30,20,75,46,12,82,57,43,69,133,182


## Pass Types

In [35]:
high_pass_types = ['High pass', 'Cross', 'Launch']
# Define low pass types based on exclusion
all_pass_types = passes_df['subEventName'].unique()
low_pass_types = [pt for pt in all_pass_types if pt not in high_pass_types and pt != 'Smart pass']

# Group by 'uniqueTeamId' and count passes
pass_counts = passes_df.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
pass_counts['Total_Passes'] = pass_counts.sum(axis=1)

# Calculate high pass and low pass totals
pass_counts['High_pass_Total'] = pass_counts[high_pass_types].sum(axis=1)
pass_counts['Low_pass_Total'] = pass_counts[low_pass_types].sum(axis=1)

# Calculate overall high pass and low pass ratios
pass_counts['High_pass_Ratio_Total'] = pass_counts['High_pass_Total'] / pass_counts['Total_Passes']
pass_counts['Low_pass_Ratio_Total'] = pass_counts['Low_pass_Total'] / pass_counts['Total_Passes']

# Now, we'll prepare the final DataFrame with the required columns
final_columns = ['Total_Passes', 'High_pass_Ratio_Total', 'Low_pass_Ratio_Total']
final_pass_types_df = pass_counts[final_columns]

# If needed, calculate zone-specific ratios (demonstrated for High pass, replicate for Low pass as necessary)
zones = passes_df['Zone'].unique()
for zone in zones:
    zone_specific_passes = passes_df[passes_df['Zone'] == zone]
    zone_pass_counts = zone_specific_passes.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
    zone_pass_counts['Total_Passes'] = zone_pass_counts.sum(axis=1)
    
    # High pass total and ratio for the zone
    zone_pass_counts['High_pass_Total_Zone'] = zone_pass_counts[high_pass_types].sum(axis=1)
    final_pass_types_df[f'High_pass_Ratio_Zone{zone}'] = zone_pass_counts['High_pass_Total_Zone'] / zone_pass_counts['Total_Passes']
    
    # Low pass total and ratio for the zone
    zone_pass_counts['Low_pass_Total_Zone'] = zone_pass_counts[low_pass_types].sum(axis=1)
    final_pass_types_df[f'Low_pass_Ratio_Zone{zone}'] = zone_pass_counts['Low_pass_Total_Zone'] / zone_pass_counts['Total_Passes']

<ipython-input-35-82b2eb0fdc0c>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_pass_types_df[f'High_pass_Ratio_Zone{zone}'] = zone_pass_counts['High_pass_Total_Zone'] / zone_pass_counts['Total_Passes']
<ipython-input-35-82b2eb0fdc0c>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_pass_types_df[f'Low_pass_Ratio_Zone{zone}'] = zone_pass_counts['Low_pass_Total_Zone'] / zone_pass_counts['Total_Passes']
<ipython-input-35-82b2eb0fdc0c>:31: SettingWithCopyWarning: 
A value is trying to be set on 

In [36]:
final_pass_types_df

subEventName,Total_Passes,High_pass_Ratio_Total,Low_pass_Ratio_Total,High_pass_Ratio_Zone2,Low_pass_Ratio_Zone2,High_pass_Ratio_Zone1,Low_pass_Ratio_Zone1,High_pass_Ratio_Zone3,Low_pass_Ratio_Zone3
uniqueTeamId,,,,,,,,,
10531---2499913,464,0.114224,0.872845,0.100000,0.890909,0.119565,0.880435,0.150000,0.783333
10531---2499927,445,0.121348,0.871910,0.080357,0.910714,0.151685,0.848315,0.209302,0.767442
10531---2499937,293,0.191126,0.788396,0.103175,0.857143,0.226277,0.773723,0.400000,0.566667
10531---2499945,459,0.137255,0.856209,0.090909,0.904762,0.184932,0.815068,0.182927,0.792683
10531---2499957,264,0.178030,0.818182,0.128713,0.861386,0.230769,0.769231,0.152174,0.847826
...,...,...,...,...,...,...,...,...,...
756---2565880,448,0.214286,0.765625,0.122449,0.867347,0.204545,0.795455,0.375000,0.566667
756---2565892,383,0.177546,0.812010,0.112299,0.871658,0.136752,0.863248,0.392405,0.594937
756---2565904,384,0.270833,0.713542,0.135338,0.849624,0.301136,0.698864,0.440000,0.506667


# Pass_Connectivity

In [37]:
from collections import defaultdict
import networkx as nx

In [38]:
def calculate_pass_connectivity(group):
    # Initialize two empty graphs for the two teams in the match
    team_graphs = {team_id: nx.Graph() for team_id in group['uniqueTeamId'].unique()}
    
    # Track the previous player and team to identify consecutive passes
    prev_player = None
    prev_team = None
    
    for i, pass_ in group.iterrows():
        current_player = pass_['playerId']
        current_team = pass_['uniqueTeamId']
        
        # Check for consecutive passes within the same team
        if prev_team == current_team and prev_player is not None:
            team_graphs[current_team].add_edge(prev_player, current_player)
        
        # Update the previous player and team for the next iteration
        prev_player = current_player
        prev_team = current_team
    
    # Calculate connectivity for each team graph
    connectivities = {}
    for team_id, graph in team_graphs.items():
        if len(graph) == 0:  # Handle cases where a team made no passes or only incomplete passes
            connectivity = 0
        else:
            connectivity = nx.average_node_connectivity(graph)
        connectivities[team_id] = connectivity
    
    return connectivities

In [39]:
test_dfdf = passes_df.iloc[:1000]

In [40]:
connectivity_results = [calculate_pass_connectivity(group) for _, group in passes_df.groupby(['matchId'])]

In [41]:
connectivity_results

[{'1619---2499738': 8.371794871794872, '1633---2499738': 6.551282051282051},
 {'1633---2499909': 6.9010989010989015, '1659---2499909': 8.14102564102564},
 {'1651---2499910': 6.065934065934066, '1610---2499910': 7.450549450549451},
 {'1628---2499911': 6.725274725274725, '1609---2499911': 8.064102564102564},
 {'1639---2499912': 7.9010989010989015, '1673---2499912': 8.131868131868131},
 {'10531---2499913': 8.571428571428571, '1612---2499913': 7.923076923076923},
 {'1646---2499914': 7.294871794871795, '1611---2499914': 9.243589743589743},
 {'1625---2499915': 8.406593406593407, '1613---2499915': 4.758241758241758},
 {'1619---2499916': 7.3076923076923075, '1624---2499916': 8.571428571428571},
 {'1644---2499917': 6.615384615384615, '1631---2499917': 6.483516483516484},
 {'1627---2499918': 6.758241758241758, '1623---2499918': 6.285714285714286},
 {'1659---2499919': 7.208791208791209, '1623---2499919': 8.582417582417582},
 {'1610---2499920': 6.857142857142857, '1639---2499920': 7.85897435897435

In [42]:
connectivity_df = [(k, v) for d in connectivity_results for k, v in d.items()]
connectivity_df = pd.DataFrame(connectivity_df, columns=['uniqueTeamId', 'Connectivity'])

In [43]:
connectivity_df.set_index('uniqueTeamId', inplace=True)
connectivity_df

,Connectivity
uniqueTeamId,
1619---2499738,8.371795
1633---2499738,6.551282
1633---2499909,6.901099
1659---2499909,8.141026
1651---2499910,6.065934
...,...
3158---2576336,8.285714
3164---2576337,8.333333
3204---2576337,8.410256


In [44]:
final_df = pd.concat([motif_ratios, final_pass_types_df, pass_direction_df, shot_df_final, connectivity_df], axis=1)
# Iterate through each column in final_df to find and adjust the specified columns
for col in final_df.columns:
    # Check if the column name contains 'Backward', 'Forward', or 'Side'
    if any(keyword in col for keyword in ['Backward', 'Forward', 'Side']):
        # Divide the column by the 'Total_Passes' column
        final_df[col] = final_df[col] / final_df['Total_Passes']
        # Rename the column to add " Ratio" at the end
        final_df.rename(columns={col: col + " Ratio"}, inplace=True)

In [45]:
final_df

,ABAB,ABCA,ABCB,ABCD,Total_Passes,High_pass_Ratio_Total,Low_pass_Ratio_Total,High_pass_Ratio_Zone2,Low_pass_Ratio_Zone2,High_pass_Ratio_Zone1,...,Side Pass_Zone_2 Ratio,Side Pass_Zone_3 Ratio,Backward Pass Ratio,Forward Pass Ratio,Side Pass Ratio,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio,Near Shots Ratio,Connectivity
uniqueTeamId,,,,,,,,,,,,,,,,,,,,,
10531---2499913,0.030369,0.123644,0.121475,0.724512,464,0.114224,0.872845,0.100000,0.890909,0.119565,...,0.221983,0.077586,0.181034,0.329741,0.489224,6.0,0.166667,0.333333,0.500000,8.571429
10531---2499927,0.018100,0.101810,0.104072,0.776018,445,0.121348,0.871910,0.080357,0.910714,0.151685,...,0.242697,0.060674,0.139326,0.357303,0.503371,9.0,0.111111,0.666667,0.222222,6.736264
10531---2499937,0.024138,0.096552,0.100000,0.779310,293,0.191126,0.788396,0.103175,0.857143,0.226277,...,0.211604,0.051195,0.156997,0.361775,0.481229,7.0,0.428571,0.000000,0.571429,6.879121
10531---2499945,0.043860,0.103070,0.100877,0.752193,459,0.137255,0.856209,0.090909,0.904762,0.184932,...,0.233115,0.089325,0.189542,0.337691,0.472767,7.0,0.000000,0.714286,0.285714,7.648352
10531---2499957,0.022989,0.065134,0.065134,0.846743,264,0.178030,0.818182,0.128713,0.861386,0.230769,...,0.151515,0.090909,0.166667,0.352273,0.481061,3.0,0.000000,0.666667,0.333333,6.560606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565880,0.029213,0.110112,0.112360,0.748315,448,0.214286,0.765625,0.122449,0.867347,0.204545,...,0.223214,0.165179,0.138393,0.314732,0.546875,19.0,0.210526,0.421053,0.368421,6.615385
756---2565892,0.036842,0.113158,0.105263,0.744737,383,0.177546,0.812010,0.112299,0.871658,0.136752,...,0.240209,0.125326,0.140992,0.352480,0.506527,11.0,0.545455,0.272727,0.181818,7.362637
756---2565904,0.031496,0.089239,0.089239,0.790026,384,0.270833,0.713542,0.135338,0.849624,0.301136,...,0.148438,0.111979,0.179688,0.346354,0.473958,14.0,0.000000,0.642857,0.357143,8.089744


In [46]:
table_name = "final_in_poss_df"

if half is not None:
    final_df.to_csv(f'{table_name}_{half}.csv', index=True)
else:
    final_df.to_csv(f'{table_name}.csv', index=True)

In [47]:
half

'H2'

# END OF FILE